## Ad Auctions for LLMs via Retrieval Augmented Genearation

Here is the outline of the experiments we plan to run on this notebook:

1. Given a query $x$, we generate fictional advertisers $\text{ad}_1, \text{ad}_2, ..., \text{ad}_n$ with their ads.
2. We sample bids $b_1, b_2, ..., b_n$ from a lognormal distribution (mean and std of this should be adjusted w.r.t. values of $q_i$)
3. We run different mechanism for this LLM query/ads configuration, *i.e.*, $(x, \{ad_i\}_{i=1}^{n}, \{b_i\}_{i=1}^{n}$.

    + single-allocation segment auction with replacement
    + single-allocation segment auction without replacement
    + multi-allocation greedy mechanism
    + naive (i) mechanism where $y = y_{\text{orig}} + \text{ad}_1 + \text{ad}_2 + ... + \text{ad}_k$
    + naive (ii) mechanism where we run 2nd price auction in each round

4. We run $N$ times each of the above mechanisms as they are randomized, we report the expectation of the following metrics:
    + social welfare (sum of $v_{i_t} q_{i_t}$)
    + revenue (sum of $p_{i_t}$)
    + relevance (sum of $q_{i_t}$)
    + output quality; we think of the following metrics
        + distance of output including ad to the original output -- measures if the output including ad answers the query or not.
        + we ask an off-the-shelf LLM to evaluate the output including ads has too much advertisement or not, probably asking it to rate from $1$ to $10$.
        + we ask an off-the-shelf LLM to evaluate paragraphs are coherent or not, probably asking it to rate from $1$ to $10$.


There are several design choices that we consider:
+ How long each segment should be, we first run with a paragraph.
+ dependant/independent segment auction
+ distribution that $x_i$ (allocation vector) comes from. Since we use RAG-based aggregation function on bids to get the allocation vector, we need to control how this allocation vector looks like, *e.g.*, we were thinking of putting some ratio for $\frac{x_{\max}}{x_{\min}}$.
+ how many segments should we consider? we put $k=3$ in our experiments, this further gives us the results for $k = 1, 2$.
+ design choices for multi-allocation greedy mechanism.

### GPT-4 API Access

In [ ]:
from openai import OpenAI




def query_to_chatgpt(prompt):
    chat_completion_det = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": prompt
            },
            {
                "role": "assistant",
                "content": ""
            }
        ],
        model = "gpt-4-turbo",
        logprobs = False,
        temperature = 1,
        max_tokens = 200
    )
    return chat_completion_det.choices[0].message.content

### Libraries

In [ ]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import time
import numpy as np
import json
from itertools import combinations_with_replacement

### Models

In [ ]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

### Initiating query and advertisers
I borrow samples from the previous experiments we did -- we may need to generate fictional advertisers (ask Sebastien).

In [1]:
prompt = 'How to activate Internationl Roaming?'
advertisers = [
    'AT&T', 'T-Mobile', 'Vodafone', 'Huawei',
    'Apple', 'Samsung', 'LG', 'Sony',
    'BMW', 'Costco', 'Starbucks', 'ALDI', 'Lidl',
]
prompt_for_adv = 'Give me a paragraph about {} so that I can use it to advertise this brand.'

### Obtaining $\text{ad}_i$ For All Advertisers

Here, we use the `prompt_for_adv` above to get a paragraph describing each of the advertisers.
Responses will generate a set of advertisements/documents that are used according to RAG to include advertisement within the output. Similar to what we had in paper denoted by: 
$$\{\text{ad}_1, \text{ad}_2, ..., \text{ad}_n\}.$$

In [ ]:
%%time
ads = []
for c in tqdm(advertisers):
    ads.append(query_to_chatgpt(prompt_for_adv.format(c)))

### Segment-wise RAG-based Output Modification

In [2]:
prompt_to_init_answer = '''
{} please respond to this question for only 1 paragraph while also advertise {} with this context:  "{}" 
Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_continue_answer = '''
please answer the question "{}" by continuing this response "{}". please write only 1 paragraph while also advertise {} with this context:  "{}" 
Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output. Ads should be minimal.
Your output must be 1 paragraph.
'''

prompt_to_init_multi_ad_k3 = '''
{} please respond to this question for only 1 paragraph while minimally advertise {}, {}, and {} with these three contexts:  1) "{}" \n 2) "{}" \n 3) "{}".
Make sure that you advertise all of them and connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_continue_multi_ad_k3 = '''
please answer the question "{}" by continuing this response "{}". write only 1 paragraph while minimally advertise {}, {}, and {} with these three contexts:  1) "{}" \n 2) "{}" \n 3) "{}".
Make sure that you advertise all of them and connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_init_multi_ad_k2 = '''
{} please respond to this question for only 1 paragraph while minimally advertise {} and {} with these two contexts:  1) "{}" \n 2) "{}".
Make sure that you advertise both of them and connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_continue_multi_ad_k2 = '''
please answer the question "{}" by continuing this response "{}". write only 1 paragraph while minimally advertise {} and {} with these two contexts:  1) "{}" \n 2) "{}".
Make sure that you advertise both of them and connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_init_multi_ad_k1 = '''
{} please respond to this question for only 1 paragraph while also advertise {} with this context:  "{}" 
Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output.
Your output must be 1 paragraph.
'''

prompt_to_continue_multi_ad_k1 = '''
{} please respond to this question while also advertise {} with this context:  "{}" 
Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
Focus on answering the question, there shouldn't be too much advertisment in the output. Ads should be minimal.
Your output must be 1 paragraph.
'''

### Segment-wise Generation
At each segment, we ask LLM to continue its response for prompt $x$ while including ad from the advertiser who has won this round of auction.

In [ ]:
def segment_based_RAG_generation(prompt: str, advertiser: str, ad: str, curr_y=None):
    if curr_y is None:
        curr_y = query_to_chatgpt(prompt_to_init_answer.format(prompt, advertiser, ad))
    else:
        curr_y += '\n' + query_to_chatgpt(prompt_to_continue_answer.format(prompt, curr_y, advertiser, ad))
    return curr_y

In [ ]:
def segment_based_multi_ad_generation(prompt: str, advertisers: list, ads: str, curr_y=None):
    if len(advertisers) == 1:
        if curr_y is None:
            curr_y = query_to_chatgpt(prompt_to_init_multi_ad_k1.format(prompt, advertisers[0], ads[0]))
        else:
            curr_y += '\n' + query_to_chatgpt(prompt_to_init_multi_ad_k1.format(prompt, curr_y, advertisers[0], ads[0]))
        
    elif len(advertisers) == 2:
        if curr_y is None:
            curr_y = query_to_chatgpt(prompt_to_init_multi_ad_k2.format(prompt, advertisers[0], advertisers[1], ads[0], ads[1]))
        else:
            curr_y += '\n' + query_to_chatgpt(prompt_to_init_multi_ad_k2.format(prompt, curr_y, advertisers[0], advertisers[1], ads[0], ads[1]))
    elif len(advertisers) == 3:
        if curr_y is None:
            curr_y = query_to_chatgpt(prompt_to_init_multi_ad_k2.format(prompt, advertisers[0], advertisers[1], advertisers[2], ads[0], ads[1], ads[2]))
        else:
            curr_y += '\n' + query_to_chatgpt(prompt_to_init_multi_ad_k2.format(prompt, curr_y, advertisers[0], advertisers[1], advertisers[2], ads[0], ads[1], ads[2]))
    else:
        assert 1 == 0
        
    return curr_y    

### Relevance Measure

Below code finds $\text{P}_{\eta}\left[\text{ad}_i\ |\ x\right]$ using embedding space of `SentenceTransformer`.

In [ ]:
def rag_based_relevance(x: str, ads: list):
    pass

### Metrics

We compare different mechanisms w.r.t below metrics. For an auction with $k$ segments, they are defined as follows:
$$\begin{align*}
    \texttt{Revenue}:& \sum_{t=1}^{k} p_{i_t} \\
    \texttt{Social Welfare}:& \sum_{t=1}^{k} v_{i_t} q_{i_t} \\
    \texttt{Relevance}:& \sum_{t=1}^{k} q_{i_t} \\
\end{align*}$$


In [3]:
SOCIAL_WELFARE = 'social_welfare'
REVENUE = 'revenue'
RELEVANCE = 'relevance'
OUTPUT = 'output'

In [4]:
def init_metrics():
    return {
        SOCIAL_WELFARE: [],
        REVENUE: [],
        RELEVANCE: [],
        OUTPUT: [],
    }

def update_metrics(metrics: dict, payment: float, value: float, rel: float, output: str):
    metrics[REVENUE].append(payment)
    metrics[SOCIAL_WELFARE].append(value * rel)
    metrics[RELEVANCE].append(rel)
    metrics[OUTPUT].append(output)

### Running Auction

We adjust bids $\{b_i\}_{i=1}^{n}$ according to scores coming from RAG, *i.e.*, $q_i$. This is implemented using pages 4 and 5 of the draft.

In [ ]:
def randomized_selection(q: np.ndarray, b: np.ndarray):
    adjusted_bids = (q * b) / np.dot(q, b)
    n = b.shape[0] # number of ads
    
    k , u = None, np.random.uniform()
    
    for i in range(n):
        W_i = (np.sum(adjusted_bids[: i]), np.sum(adjusted_bids[:, i+1]))
        if W_i[0] <= u <= W_i[1]:
            k = i
    
    all_other_than_k = np.sum(adjusted_bids) - adjusted_bids[k]
    aux_value = (np.sum(adjusted_bids[:k])) / all_other_than_k
    
    if aux_value >= u:
        payment = (np.sum(adjusted_bids[:k]) - u * all_other_than_k) / (u * q[k])
    else:
        payment = (np.sum(adjusted_bids[:k]) - u * all_other_than_k) / ((u - 1) * q[k])
    
    return k, payment
        

### Mechanisms
+ single-allocation segment auction with replacement
+ single-allocation segment auction without replacement
+ multi-allocation greedy mechanism
+ naive (i) mechanism where $y = y_{\text{orig}} + \text{ad}_1 + \text{ad}_2 + ... + \text{ad}_k$
+ naive (ii) mechanism where we run 2nd price auction in each round

For each of these mechanisms, we need to have a set of bids $\{b_i\}_{i=1}^{n}$ from all advertisers, as well as RAG-based relevancy metric $q_i^{(t)}$. We compute RAG-based relevancy by measuring the similarity of current generated response (or query $x$) with ads (documents). Mechanism is run for $k$ iterations. We store metrics meanwhile that are later reported as properties of different mechanisms.

In [ ]:
def single_allocation_with_replacement(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    dependent: bool = False):
    
    curr, v = '', bids
    metrics = init_metrics()
    
    for t in range(num_of_segments):
        q = rag_based_relevance(prompt + '\n' + curr if dependent else prompt, ads, )
        k, payment = randomized_selection(q, bids)
        # curr = segment_based_RAG_generation(prompt=prompt, advertiser=advertisers[k], ad=ads[k], curr_y=curr)
        # TODO: We ignore abobe line for now as we only care about metrics irrelevant to output.
        update_metrics(metrics, payment, v[k], q[k], curr)
    
    return metrics, curr
            
    

In [ ]:
def single_allocation_without_replacement(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    dependent: bool = False):
    
    curr, v = '', bids
    metrics = init_metrics()
    selected_ads = np.zeros(n) # keep track of ads that are selected in previous rounds of auction.
    
    for t in range(num_of_segments):
        q = rag_based_relevance(prompt + '\n' + curr if dependent else prompt, ads, )
        k, payment = randomized_selection(q, bids)
        assert selected_ads[k] == 0 # shouldn't have been selected before.
        # curr = segment_based_RAG_generation(prompt=prompt, advertiser=advertisers[k], ad=ads[k], curr_y=curr)
        # TODO: We ignore abobe line for now as we only care about metrics irrelevant to output.
        selected_ads[k] = 1 # k is winner
        bids[k] = 0 # never gonna be winner again
        update_metrics(metrics, payment, v[k], q[k], curr)        
    
    return metrics, curr
            
    

In [ ]:
def single_allocation_naive_i(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    dependent: bool = False):
    
    curr, v = '', bids
    metrics = init_metrics()
    
    for t in range(num_of_segments):
        q = rag_based_relevance(prompt + '\n' + curr if dependent else prompt, ads, )
        k, payment = randomized_selection(q, bids)
        curr = curr + ads[k]
        update_metrics(metrics, payment, v[k], q[k], curr)
        
    return metrics, curr
    

In [ ]:
def single_allocation_naive_ii(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    dependent: bool = False):
    
    curr, v = '', bids
    metrics = init_metrics()

    for t in range(num_of_segments):
        q = rag_based_relevance(prompt + '\n' + curr if dependent else prompt, ads, )
        k, payment = randomized_selection(np.ones(bids.shape[0]), bids) # removing the effect of RAG -- second price auction
        # curr = segment_based_RAG_generation(prompt=prompt, advertiser=advertisers[k], ad=ads[k], curr_y=curr)
        # TODO: We ignore abobe line for now as we only care about metrics irrelevant to output.
        update_metrics(metrics, payment, v[k], q[k], curr)
        
    return metrics, curr
    

In [ ]:
prompt_to_add_adv_k_2 = '''
{} please respond to this question for 1 paragraph while minimally advertise {} and {} with these two contexts:  1) "{}" \n 2) "{}".
Make sure that you advertise both of them and connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
'''

prompt_to_add_adv_k_1 = '''
{} please respond to this question for 1 paragraph while also advertise {} with this context:  "{}". Make sure to connect the answer and the advertisement very naturally, not something like appending the ads after just answering the question.
'''

In [ ]:
def get_relevency(y: str, A: list, ads: list):
    y_embedding = model.encode(y)
    ads_embedding = model.encode([ads[ad] for ad in A])
    return util.dot_score(y_embedding, ads_embedding)[0].numpy()
    

In [ ]:
def multi_allocation_greedy(
    prompt: str,
    advertisers: list,
    ads: list, 
    bids: np.ndarray,
    num_of_segments: int,
    num_of_ads_in_each_segment: int):
    
    curr, v, n = '', bids, bids.shape[0]
    metrics = init_metrics()
    
    
    
    for t in range(num_of_segments):
        A = []
        
        while len(A) < num_of_ads_in_each_segment:
            adjusted_bids = np.zeros(n)
            for i in tqdm(range(n)):
                if i in A:
                    continue
            
                A_i = A + [i]
                y_A_i = segment_based_multi_ad_generation(prompt=prompt, advertisers=[advertisers[j] for j in A_i], ads=[ads[j] for j in A_i], curr_y=curr)
                
                q_A_i = get_relevency(y_A_i, A_i, ads)
                adjusted_bids[i] = np.dot(q_A_i, bids[np.array(A_i)])
                
            
            i_star = np.argmax(adjusted_bids)
            A = A + [i_star]
        
        y_A = segment_based_multi_ad_generation(prompt=prompt, advertisers=[advertisers[j] for j in A], ads=[ads[j] for j in A], curr_y=curr)
        q_A = get_relevency(y_A, A, ads)
        
        metrics[OUTPUT].append(y_A)
        metrics[SOCIAL_WELFARE].append(np.dot(q_A, bids[np.array(A)]))
        metrics[RELEVANCE].append(np.sum(q_A))
        curr = curr + '\n' + y_A
    
    return metrics, curr